In [1]:
import os
import cv2
import pandas as pd
import numpy as np

In [2]:
def is_blank_region(region, blank_threshold):
    """
    Checks if a given image region is blank (mostly white).

    Parameters:
        region (numpy array): Image region to check.
        blank_threshold (float): Proportion of white pixels to consider the region blank.

    Returns:
        bool: True if the region is blank, False otherwise.
    """
    # Convert to grayscale if it's a color image
    if len(region.shape) == 3:
        region = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)

    # Count white pixels (assume white is near 255)
    white_pixels = np.sum(region >= 64)
    total_pixels = region.size
    return (white_pixels / total_pixels) >= blank_threshold


In [3]:
def expand_bounding_box(image, bbox, left_expand, right_expand, vertical_region_height, step, blank_threshold):
    """
    Expands a bounding box horizontally and vertically based on detecting blank regions.

    Parameters:
        image (numpy array): Input music score image.
        bbox (list): Bounding box [x_min, y_min, x_max, y_max].
        horizontal_expand (int): Pixels to expand horizontally.
        vertical_region_height (int): Fixed height for the region used in vertical blank detection.
        step (int): Step size for moving the vertical region.
        blank_threshold (float): Proportion of white pixels to consider a region blank.

    Returns:
        list: Expanded bounding box [x_min, y_min, x_max, y_max].
    """
    x_min, y_min, x_max, y_max = bbox
    height, width = image.shape[:2]

    # Horizontal expansion (fixed amount)
    x_min_expanded = max(0, x_min - left_expand)
    x_max_expanded = min(width, x_max + right_expand)

    # Vertical expansion using blank detection
    # Check upward
    for offset in range(0, height, step):  # Move the region upwards in steps
        new_y_min = max(0, y_min - offset - vertical_region_height)
        region = image[new_y_min:new_y_min + vertical_region_height, x_min:x_max]
        if is_blank_region(region, blank_threshold):
            y_min = new_y_min
            break

    # Check downward
    for offset in range(0, height, step):  # Move the region downwards in steps
        new_y_max = min(height, y_max + offset + vertical_region_height)
        region = image[new_y_max - vertical_region_height:new_y_max, x_min:x_max]
        if is_blank_region(region, blank_threshold):
            y_max = new_y_max
            break

    return [x_min_expanded, y_min, x_max_expanded, y_max]


In [4]:
def draw_bounding_boxes(output_csv, image_folder, output_folder, left_expand=65, right_expand=55, vertical_region_height=55,
                                    step=5, blank_threshold=0.97, box_color=(0, 0, 255), thickness=3):
    """
    Draws bounding boxes on music scores based on the CSV table, with vertical expansion using blank region detection.

    Parameters:
        output_csv (str): Path to the CSV file containing bounding boxes.
        image_folder (str): Folder containing the corresponding raw music score images.
        output_folder (str): Folder to save the output images with bounding boxes drawn.
        horizontal_expand (int): Pixels to expand horizontally.
        vertical_region_height (int): Fixed height of the region for vertical blank detection.
        step (int): Step size for moving the vertical region.
        blank_threshold (float): Proportion of white pixels to consider a region blank.
        box_color (tuple): Color of the bounding boxes (default: red in BGR format).
        thickness (int): Thickness of the bounding box lines.
    """
    
    data = pd.read_csv(output_csv)               # Load the CSV file
    os.makedirs(output_folder, exist_ok=True)    # Ensure the output folder exists
    grouped = data.groupby("File Name")          # Group rows by file name

    for file_name, group in grouped:             # Iterate through the images
        
        image_path = os.path.join(image_folder, file_name)
        if not os.path.exists(image_path):
            print(f"Image file {file_name} not found in {image_folder}. Skipping...")
            continue

        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to read image {image_path}. Skipping...")
            continue

        for _, row in group.iterrows():    # Iterate through the slurs on each page
            
            bbox = [int(float(coord)) for coord in row["BoundingBox"].strip("[]").split(", ")]
            expanded_bbox = expand_bounding_box(image, bbox, left_expand, right_expand, vertical_region_height, step, blank_threshold)
            x_min, y_min, x_max, y_max = expanded_bbox
            # Add expanded bounding box
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), box_color, thickness)

        # Save the image with bounding boxes
        output_path = os.path.join(output_folder, file_name)
        cv2.imwrite(output_path, image)
        print(f"Saved: {output_path}")


In [5]:
output_csv = "output_objects.csv"
image_folder = "raw"
output_folder = "expand_bbox"
draw_bounding_boxes(output_csv, image_folder, output_folder, left_expand=65, right_expand=55, vertical_region_height=55)

Saved: expand_bbox\HN7_128.png
Saved: expand_bbox\HN7_129.png
Saved: expand_bbox\HN7_130.png
Saved: expand_bbox\HN7_131.png
Saved: expand_bbox\HN7_132.png
Saved: expand_bbox\HN7_133.png
Saved: expand_bbox\HN7_134.png
Saved: expand_bbox\HN7_135.png
Saved: expand_bbox\HN7_136.png
Saved: expand_bbox\HN7_137.png
Saved: expand_bbox\HN7_138.png
Saved: expand_bbox\HN7_139.png
Saved: expand_bbox\HN7_140.png
Saved: expand_bbox\HN7_141.png
Saved: expand_bbox\HN7_142.png
Saved: expand_bbox\HN7_144.png
Saved: expand_bbox\HN7_145.png
Saved: expand_bbox\HN7_146.png
Saved: expand_bbox\HN7_147.png
Saved: expand_bbox\HN7_148.png
Saved: expand_bbox\HN7_149.png
Saved: expand_bbox\HN7_150.png
Saved: expand_bbox\HN7_151.png
Saved: expand_bbox\HN7_153.png
Saved: expand_bbox\HN7_154.png
Saved: expand_bbox\HN7_155.png
Saved: expand_bbox\HN7_156.png
Saved: expand_bbox\HN7_157.png
Saved: expand_bbox\HN7_158.png
Saved: expand_bbox\HN7_159.png
Saved: expand_bbox\HN7_160.png
Saved: expand_bbox\HN7_161.png
Saved: e

In [6]:
image_folder = "raw"
file_name = "HN7_128.png"
image_path = os.path.join(image_folder, file_name)
image = cv2.imread(image_path)
print(image.shape)

(6000, 4500, 3)
